<a href="https://colab.research.google.com/github/krcpr007/iris_encryption-/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
from PIL import Image
import os
import numpy as np
image_folder = '/content/gdrive/MyDrive/iris/train/eye'
images = []
for filename in os.listdir(image_folder):
    img = Image.open(os.path.join(image_folder, filename)).convert('L') # convert to grayscale
    img = img.resize((64, 64)) # resize image to 64x64 pixels
    images.append(img)

In [ ]:
print(images)

[<PIL.Image.Image image mode=L size=64x64 at 0x7F75F44B3E20>, <PIL.Image.Image image mode=L size=64x64 at 0x7F76905785B0>, <PIL.Image.Image image mode=L size=64x64 at 0x7F76902BFFA0>, <PIL.Image.Image image mode=L size=64x64 at 0x7F75F4854940>, <PIL.Image.Image image mode=L size=64x64 at 0x7F75F45AF8B0>, <PIL.Image.Image image mode=L size=64x64 at 0x7F760B73A400>, <PIL.Image.Image image mode=L size=64x64 at 0x7F76902BFEB0>, <PIL.Image.Image image mode=L size=64x64 at 0x7F76902BFF70>, <PIL.Image.Image image mode=L size=64x64 at 0x7F7690578490>, <PIL.Image.Image image mode=L size=64x64 at 0x7F75F52C8AC0>, <PIL.Image.Image image mode=L size=64x64 at 0x7F769053EAF0>, <PIL.Image.Image image mode=L size=64x64 at 0x7F7613574D00>, <PIL.Image.Image image mode=L size=64x64 at 0x7F76902CCD90>, <PIL.Image.Image image mode=L size=64x64 at 0x7F760870E040>, <PIL.Image.Image image mode=L size=64x64 at 0x7F75EF9FA9A0>, <PIL.Image.Image image mode=L size=64x64 at 0x7F76902CCDF0>, <PIL.Image.Image image 

In [ ]:
from sklearn.preprocessing import StandardScaler

# convert images to numpy arrays
X = []
for img in images:
    X.append(np.array(img).flatten())
X = np.array(X)

# normalize dataset
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)
# print(X_normalized)

In [ ]:
from tensorflow.keras import layers, models
import tensorflow as tf
import keras
# define CNN model
model = models.Sequential([
    layers.Reshape((64, 64, 1), input_shape=(4096,)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(3, activation='softmax')
])   

# compile model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# train model
y = np.array([0] * 50 + [1] * 50 + [2] * 50) # set target labels
y_onehot = tf.one_hot(y, depth=3) # one-hot encode target labels
X_normalized = X_normalized[:150] # taking only 150 
model.fit(X_normalized, y_onehot, epochs=10) #model fit 
# print('Number of images loaded:', len(X_normalized)) #328
# print('Number of labels:', y_onehot.shape[0])  #150
# keras.utils.plot_model(model, show_shapes=True)
test_loss, test_acc = model.evaluate(X_normalized, y_onehot)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

Epoch 1/10
5/5 [==============================] - 3s 186ms/step - loss: 1.1849 - accuracy: 0.3400
Epoch 2/10
5/5 [==============================] - 1s 250ms/step - loss: 1.0552 - accuracy: 0.4267
Epoch 3/10
5/5 [==============================] - 2s 367ms/step - loss: 1.0022 - accuracy: 0.5200
Epoch 4/10
5/5 [==============================] - 4s 749ms/step - loss: 0.9476 - accuracy: 0.5800
Epoch 5/10
5/5 [==============================] - 4s 745ms/step - loss: 0.8813 - accuracy: 0.5733
Epoch 6/10
5/5 [==============================] - 3s 530ms/step - loss: 0.7965 - accuracy: 0.6467
Epoch 7/10
5/5 [==============================] - 3s 511ms/step - loss: 0.7089 - accuracy: 0.7133
Epoch 8/10
5/5 [==============================] - 3s 611ms/step - loss: 0.6296 - accuracy: 0.7600
Epoch 9/10
5/5 [==============================] - 3s 582ms/step - loss: 0.5782 - accuracy: 0.7867
Epoch 10/10
5/5 [==============================] - 1s 127ms/step - loss: 0.3145 - accuracy: 0.8867
Test loss: 0.314534

In [ ]:
# extract feature vector V1
V1 = model.predict(X_normalized)
print(V1.shape)

5/5 [==============================] - 0s 59ms/step
(150, 3)


In [ ]:
# !pip install reedsolo
from reedsolo import RSCodec

# encode feature vector V using RS code
rs = RSCodec(10)
V_encoded = rs.encode(V1.flatten())
print(V_encoded)

bytearray(b'\xff\xff\x7f?\x85\xd1\x86#t\x8a\xe11_\xff\x7f?H\x94U/\xf22 7\xff\xff\x7f?\xea\xde\xe8#\xee\xdc\x02-t\xf6\x7f?m\xa7n5\xe9\xfd\x179\x8b\xfd\x7f?[\x04\xd7-a\xd7\x1c8\xe7\xfb\x7f?%\xe3z/?\xe8\x828\xb3\xff\x7f?~p\xf9\'\xfb\x00\x966\xa0\xf1\x7f?\xc2x\xae54\xb3d9\xff\xff\x7f?\xdfc{/\xba~\xc32p\xf0\x7f?\xf3rg5\xb2,x9\xdd\xff\x7f?z\xa2\x912f\xfe\x0b6\x87\xfb\x7f?\xa1\x89\x843&\xd1\x8e8\xf9\xff\x7f?\x9a-\x9d47L54\xdd\xf8\x7f?\xce5|8{?L8\xb1\xfd\x7f?\x17]\x06+h\x87\x138\xa3\xfc\x7f?N\n\x911\xf3\xcdV8K\xfc\x7f?\x8d1k8v\xe3\xe74\xf3\xff\x7f?\x1e\xc7(1L\x07@5\xa7\xfb\x7f?\x95\xd0\x877\x8d\xddQ8_\xfb\x7f?`\xef\xae7z\xb5P8\xff\xff\x7f?\x8c{|*\x8d\xc6\x9a*L\xf6\x7f?\xf2\x04\xc68#\xf8`8\xb5\xfe\x7f?\xfch<.\x18\xa5\xa47u\xff\x7f?\x1ex\xdd-h[\t7\xff\xff\x7f?_@-0|\x85\x89.\xff\xff\x7f?%Mt,\x14e\x0e2\xd5\xfc\x7f?`\xa2\xce2\xa6{J8K\xff\x7f?@\xd0&6\xdeM\n7\xa9\xfc\x7f?Q\xa5C2ETU8\x10\xf0\x7f?\xd0\x89\x886\xe7\xa2z9C\xff\x7f?|_\x035\xff\xd337\xa9\xfe\x7f?{^94\xe4\x81\xa97\xff\xff\x7f?\xe10\x0c.tM\x

In [ ]:
# encrypt image using XOR operation
img = Image.open(os.path.join(image_folder, '1-556.JPG')).convert('L') # load example image
img = img.resize((64, 64)) # resize image to 64x64 pixels
X = np.array(img).flatten() # convert image to numpy array
X_normalized = scaler.transform(X.reshape(1, -1)) # normalize image

V2 = model.predict(X_normalized) # extract feature vector V2
V_encoded = rs.encode(V2.flatten()) # encode feature vector V2 using RS code
key = V_encoded
print(key)

1/1 [==============================] - 0s 173ms/step
bytearray(b'\xb9\xfc|?&\xc4{;\xac\xe0\x01<Z+\xff\xa9\x8a\xad\xf4\xcf`\x8e')


In [ ]:
img = Image.open(os.path.join(image_folder, '1-556.JPG')).convert('L') # load example image
img = img.resize((64, 64)) # resize image to 64x64 pixels
X = np.array(img).flatten() # convert image to numpy array

# reshape key to match the shape of X
key_reshaped = np.tile(key, len(X) // len(key) + 1)[:len(X)]

# encrypt the image using XOR operation
X_encrypted = bytearray(X ^ key_reshaped)

# save the encrypted image
img_encrypted = Image.frombytes('L', (64, 64), bytes(X_encrypted))
img_encrypted.save(os.path.join(image_folder, 'Rajan.jpg'))
